In [1]:
import gym
import ptan
import random
import numpy as np

import torch
import torch.optim as optim

from ignite.engine import Engine

from lib import common, dqn_extra

NAME = "06_dueling"
STATES_TO_EVALUATE = 1000
EVAL_EVERY_FRAME = 100

In [3]:
@torch.no_grad()
def evaluate_states(states, net, device, engine):
    s_v = torch.tensor(states).to(device)
    adv, val = net.adv_val(s_v)
    engine.state.metrics['adv'] = adv.mean().item()
    engine.state.metrics['val'] = val.mean().item()

In [4]:
random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("PongNoFrameskip-v4")
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [12]:
net = dqn_extra.DuelingDQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device = device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

In [15]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model,
                                  gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % EVAL_EVERY_FRAME == 0:
        eval_states = getattr(engine.state, "eval_states", None)
        if eval_states is None:
            eval_states = buffer.sample(STATES_TO_EVALUATE)
            eval_states = [np.array(transition.state, copy=False) for transition in eval_states]
            eval_states = np.array(eval_states, copy=False)
            engine.state.eval_states = eval_states
        evaluate_states(eval_states, net, device, engine)
    return {
        "loss" : loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine = Engine(process_batch)
common.setup_ignite(engine, params, exp_source, NAME, extra_metrics=("adv", "val"))
engine.run(common.batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-19, steps=929, speed=0.0 f/s, elapsed=0:01:00
Episode 2: reward=-21, steps=968, speed=0.0 f/s, elapsed=0:01:00
Episode 3: reward=-19, steps=1020, speed=0.0 f/s, elapsed=0:01:00
Episode 4: reward=-20, steps=896, speed=0.0 f/s, elapsed=0:01:00
Episode 5: reward=-19, steps=1066, speed=0.0 f/s, elapsed=0:01:00
Episode 6: reward=-19, steps=935, speed=0.0 f/s, elapsed=0:01:00
Episode 7: reward=-21, steps=760, speed=0.0 f/s, elapsed=0:01:00
Episode 8: reward=-21, steps=847, speed=0.0 f/s, elapsed=0:01:00
Episode 9: reward=-19, steps=969, speed=0.0 f/s, elapsed=0:01:00
Episode 10: reward=-21, steps=878, speed=0.0 f/s, elapsed=0:01:00
Episode 11: reward=-21, steps=944, speed=0.0 f/s, elapsed=0:01:00
Episode 12: reward=-21, steps=759, speed=60.0 f/s, elapsed=0:01:11
Episode 13: reward=-21, steps=819, speed=60.0 f/s, elapsed=0:01:24
Episode 14: reward=-20, steps=938, speed=60.1 f/s, elapsed=0:01:39
Episode 15: reward=-18, steps=1224, speed=60.1 f/s, elapsed=0:01:59
Episode 16: 

Engine run is terminating due to exception: .


KeyboardInterrupt: 